In [74]:
%reset

In [75]:
%whos

Interactive namespace is empty.


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
# from selenium.webdriver.common.action_chains import ActionChains
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.support.wait import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
from urllib.request import Request
from urllib.request import urlopen
import urllib

import pandas as pd
import time

In [2]:
# options = webdriver.ChromeOptions()
# driver = webdriver.Chrome(service=Service("../driver/chromedriver-linux64/chromedriver"),
#                           options=options)
driver = webdriver.Chrome(service=Service("../chromedriver-win64/chromedriver.exe"))
url = "https://www.opinet.co.kr/searRgSelect.do"
driver.get(url)

In [78]:
driver.get(url)

In [3]:
gu_option = driver.find_element(By.ID, 'SIGUNGU_NM0')
gu_list = gu_option.find_elements(By.TAG_NAME, 'option')

real_gu = []
for i in gu_list[1:]:
    real_gu.append(i.get_attribute('value'))

In [4]:
gas_stations = pd.DataFrame({
    '브랜드':[],
    '주유소명':[],
    '주소':[],
    '휘발유':[],
    '경유':[],
    '세차장':[],
    '충전소':[],
    '경정비':[],
    '편의점':[],
    '24시영업':[]
})

In [6]:
for i in real_gu:
    gu_option = driver.find_element(By.ID, 'SIGUNGU_NM0')
    gu_option.send_keys(i)
    time.sleep(0.5)

    search_result = driver.find_element(By.ID, "body1")
    search_list = search_result.find_elements(By.TAG_NAME,"tr")

    for j in range(len(search_list)):

        search_list[j].find_element(By.TAG_NAME, "a").click()
        time.sleep(0.2)
        page = driver.page_source
        soup = BeautifulSoup(page, "html.parser")
        time.sleep(1)

        total_info = soup.find(id = "os_dtail_info")
        title = total_info.find(id ="os_nm").string

        basic_info = total_info.find(class_= 'gis_detail_info_bcon')
        address = basic_info.find(id='rd_addr').string
        brand = basic_info.find(id='poll_div_nm').string

        oil_price_table = total_info.find(id = 'infoTbody')
        oil_price_table_row = oil_price_table.find_all("tr")
        gasoline = oil_price_table_row[1].find_all("td")[1].get_text()
        diesel = oil_price_table_row[2].find_all("td")[1].get_text()

        total_info = driver.find_element(By.ID, "os_dtail_info")
        add_info = total_info.find_element(By.CLASS_NAME, "service")
        service_info = add_info.find_elements(By.TAG_NAME, "img")
        add_list = []
        for i in range(len(service_info)):
            if 'off' in str(service_info[i].get_attribute("src")) :
                add_list.append('N')
            else :
                add_list.append('Y')

        target_gas_station = pd.DataFrame({
            '브랜드':[brand],
            '주유소명':[title],
            '주소':[address],
            '휘발유':[gasoline],
            '경유':[diesel],
            '세차장':[add_list[0]],
            '충전소':[add_list[1]],
            '경정비':[add_list[2]],
            '편의점':[add_list[3]],
            '24시영업':[add_list[4]]
        })
        gas_stations = pd.concat([gas_stations, target_gas_station])




In [13]:
gas_stations.reset_index(drop = True, inplace = True)

In [14]:
gas_stations.tail(30)

,브랜드,주유소명,주소,휘발유,경유,세차장,충전소,경정비,편의점,24시영업
409,SK에너지,자하문주유소,서울 종로구 자하문로 248 (부암동),"1,829원","1,699원",Y,N,N,N,N
410,S-OIL,안풍주유소,서울 종로구 자하문로 303,"1,829원","1,699원",Y,N,N,N,N
411,S-OIL,구도일주유소특종,서울 종로구 평창문화로 90 (평창동),"1,860원","1,730원",Y,N,N,N,N
412,SK에너지,(주)중앙에너비스 혜화주유소,서울 종로구 창경궁로35길 1,"2,287원","2,085원",N,N,N,N,N
413,SK에너지,SK북악주유소,서울 종로구 평창문화로 137,"2,298원","2,148원",Y,N,N,N,N
414,GS칼텍스,경복궁주유소,서울 종로구 율곡로 6 (중학동),"2,310원","2,160원",N,N,N,N,N
415,GS칼텍스,(주)대양씨앤씨 사직주유소,서울 종로구 사직로 65 (사직동),"2,320원","2,195원",N,N,N,N,N
416,S-OIL,다산주유소,서울 중구 다산로 233,"1,769원","1,609원",N,N,N,N,N
417,S-OIL,장충드림주유소,서울 중구 동호로 296,"1,825원","1,695원",Y,N,N,N,N
418,알뜰주유소,세화주유소,서울 중구 왕십리로 403,"1,828원","1,698원",Y,N,Y,N,Y
